In [2]:
import pypsa
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(1, '/mnt/d/GitHub/pypsa_utilities/eskom data/')

import data_utilities as du
import plotting_utilities as pu
import financial_utilities as fu
import xarray as xr

In [ ]:
eskom_data=du.eskom_data('../data/ESK2546.csv')

In [ ]:
df = pd.read_csv('../data/Eskom EAF data.csv',index_col=[1,0],parse_dates=True)
df['capacity'] = df['EAF %']/100 * df['MW'] 


for y in [2017,2018,2019,2020,2021,2022]:
    data=df.loc['Generation Total','PCLF %']
    plt.plot(data[data.index.year==y].values)



In [ ]:
df = pd.read_csv('../data/Eskom EAF data.csv',index_col=[1,0],parse_dates=True)
df['capacity'] = df['EAF %']/100 * df['MW'] 
stations=df[df.type=='coal'].index.get_level_values(0)

In [ ]:
stations

In [ ]:
df = pd.read_csv('../data/Eskom EAF data.csv',index_col=[1,0],parse_dates=True)
df['capacity'] = df['EAF %']/100 * df['MW'] 
stations=df[df.type=='coal'].index.get_level_values(0).unique()
eaf=pd.DataFrame(0,index=stations,columns=range(2017,2023))
planned=pd.DataFrame(0,index=stations,columns=range(2017,2023))
unplanned=pd.DataFrame(0,index=stations,columns=range(2017,2023))
for s in stations:
    eaf.loc[s,:]=df.loc[s,'EAF %'].groupby(df.loc[s,'EAF %'].index.year).mean()
    planned.loc[s,:]=df.loc[s,'PCLF %'].groupby(df.loc[s,'PCLF %'].index.year).mean()
    unplanned.loc[s,:]=df.loc[s,'UCLF %'].groupby(df.loc[s,'UCLF %'].index.year).mean()+df.loc[s,'OCLF %'].groupby(df.loc[s,'OCLF %'].index.year).mean()

In [ ]:
df = pd.read_csv('../data/Eskom EAF data.csv',index_col=[1,0],parse_dates=True)
df['capacity'] = df['EAF %']/100 * df['MW'] 
df=df[df.type=='coal']
df.index=df.index.get_level_values(1)
variable='EAF %'
type_avg=df.groupby(level=0).mean()
type_avg.drop(['MW','capacity'],axis=1)
type_avg.loc[:,variable].groupby(type_avg.index.year).mean()

In [ ]:
df = pd.read_csv('../data/Eskom EAF data.csv',index_col=[1,0],parse_dates=True)
df['capacity'] = df['EAF %']/100 * df['MW'] 
stations=df[df.type=='coal'].index.get_level_values(0)

installed_capacity=pd.DataFrame(0,index=stations,columns=range(2017,2023))
for s in stations:
    for y in range(2017,2023):
        installed_capacity.loc[s,y]=df.loc[(s,str(y)),'MW'].mean()

In [ ]:
installed_capacity.to_clipboard()

In [ ]:
mean_pclf=type_avg[variable].groupby(type_avg.index.month).mean().to_clipboard()
for y in [2017,2018,2019,2020,2021,2022]:
    plt.plot(type_avg.loc[str(y),variable].values)

plt.plot(mean_pclf.values,'k--',linewidth=3)
plt.legend([2017,2018,2019,2020,2021,2022])

In [ ]:
mean_pclf=coal_plants['UCLF %'].groupby(coal_plants.index.month).mean()
for y in [2017,2018,2019,2020,2021,2022]:
    plt.plot(coal_plants.loc[str(y),'UCLF %'].values)

plt.plot(mean_pclf.values,'k--',linewidth=3)
plt.legend([2017,2018,2019,2020,2021,2022])

In [ ]:
plt.plot(eskom_data['coal_eaf'])
plt.plot(df[df.type=='coal'].groupby('Date').sum()['capacity'].divide(df[df.type=='coal'].groupby('Date').sum()['MW']))
plt.ylabel('Coal fleet EAF')

In [212]:
n = pypsa.Network('../results/version-0.6/networks/solved_za_original_27-supply_redz_lcopt_Co2L.nc')

INFO:pypsa.io:Imported network solved_za_original_27-supply_redz_lcopt_Co2L.nc has buses, carriers, generators, global_constraints, lines, loads, storage_units


In [213]:
n.generators_t.p[n.generators[n.generators.carrier=='coal'].index].sum().sum()*n.carriers.loc['coal','co2_emissions']/1e6

27.773794507377158

In [182]:
n.generators[n.generators.carrier=='coal'].index

Index(['Kendal', 'Kusile', 'MajubaDry', 'MajubaWet', 'Matimba', 'Medupi',
       'Municipal_Coal', 'Sasol_SSF', 'RSA coal_2040'],
      dtype='object', name='Generator')

In [195]:
n.generators_t.p[n.generators[n.generators.carrier=='coal'].index].sum()

Generator
Kendal               1.474924e+07
Kusile               8.415800e+06
MajubaDry            7.534441e+06
MajubaWet            8.226581e+06
Matimba              2.132717e+07
Medupi               1.488773e+07
Municipal_Coal       1.042342e+06
Sasol_SSF            2.575943e+06
GAUTENG coal_2040    1.038940e+02
LIMPOPO coal_2040    9.774378e+01
dtype: float64

In [176]:
n.carriers.loc['coal','co2_emissions']

0.354

In [214]:
n.generators.p_nom_opt.groupby(n.generators.carrier).sum()

carrier
CCGT        1209.475642
CSP          600.000000
OCGT       24527.533363
biomass      193.060000
coal       18610.074049
diesel      1005.000000
gas          425.000000
hydro       2447.020000
nuclear     1854.063093
onwind     77881.367380
solar      42901.832989
Name: p_nom_opt, dtype: float64

In [210]:
n.storage_units.p_nom_opt.groupby(n.storage_units.carrier=='battery').sum()

carrier
False     2912.000000
True     20183.865972
Name: p_nom_opt, dtype: float64

In [155]:
n.generators_t.p_max_pu['RSA onwind_2040'].mean()

0.36794823059360743

In [207]:
n.generators_t.p['RSA solar_2040'].mean()/n.generators.p_nom_opt['RSA solar_2040']

KeyError: 'RSA solar_2040'

In [209]:
co2_emi = (n.generators_t.p.sum()/n.generators.efficiency * n.generators.carrier.map(n.carriers.co2_emissions)).sum()
co2_emi/1e6

49.99998208501157

In [160]:
n.investment_period_weightings

,objective,years
name,,
2040,1,1


In [ ]:
n.generators_t.p_max_pu.plot()